In [1]:
import numpy as np
from astropy.io import fits
import matplotlib
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import os
from ProjectF import MLAData,classification, Object,storing
import random

In [3]:
bin_size = np.str(10)
TrainingDir = os.path.normpath("D:\Data")
slash =  os.path.normpath("\\")
TrainingFolder =  os.path.normpath("\Training")
PlateDir ="D:"+slash+"Data"+slash+"FullRebin10"+".txt"
with open(PlateDir) as f:
    Spectra_Files = f.read().splitlines()

In [4]:
wav_log=[]
Full_table = []
PLATEIDs = []
for spectrum in Spectra_Files:
    plate_ = fits.open( TrainingDir+slash+'rebin'+slash+spectrum ,memmap=True)
    primhdu_ = plate_[0]
    hdu =  plate_[1].data
    Full_table.append(hdu)
    PLATEIDs.append(primhdu_.header['Plate'])
    wav_log.append(primhdu_.header['LogWav'])

In [5]:
plate_r  = []
plate_n  = []
plate_c  = []
plate_x  = []
plate_y  = []
i = 0
while i < len(PLATEIDs):
    plate_hdu  = Full_table[i]
    obj = 0
    while obj <len(plate_hdu):
        currentobj = plate_hdu[obj]
        Currentplate_z = currentobj[4]
        Currentplate_y = currentobj[2]
        Currentplate_x = currentobj[1]
        Currentplate_n = currentobj[0]
        plate_r.append(Currentplate_z)
        plate_n.append(Currentplate_n)
        plate_x.append(Currentplate_x[:800])
        plate_y.append(Currentplate_y)
        obj = obj +1
    i = i+1



In [6]:
hiddenlayer_format = (13)
backprop_method = 'lbfgs'
lr=0.00001
act =  'logistic'#'tanh'
mlp = MLPClassifier(hidden_layer_sizes=hiddenlayer_format,max_iter=500, solver = backprop_method,learning_rate_init=lr,activation=act) ##Think About

In [7]:
len(plate_x[3])

800

In [8]:
scaler = StandardScaler()  
scaler.fit(np.array(plate_x))  
plate_x = scaler.transform(plate_x)    
mlp.fit(plate_x,plate_y)

MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=13, learning_rate='constant',
       learning_rate_init=1e-05, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [9]:
TrainingDir = os.path.normpath("D:\Data")
slash =  os.path.normpath("\\")
TrainingFolder =  os.path.normpath("\Training")
PlateDir ="D:"+slash+"Data"+slash+"FullTest10.txt"
with open(PlateDir) as f:
    Spectra_TestFiles = f.read().splitlines()

In [10]:
wav_log_Test=[]
Full_table_Test = []
PLATEID_Test = []
for spectrum in Spectra_TestFiles:
    plate_test = fits.open( TrainingDir+slash+'rebintrain'+slash+spectrum ,memmap=True)
    primhdu_test = plate_[0]
    hdu =  plate_test[1].data
    Full_table_Test.append(hdu)
    PLATEID_Test.append(primhdu_.header['Plate'])
    wav_log_Test.append(primhdu_.header['LogWav'])

In [11]:
plate_rt = []
plate_nt = []
plate_ct = []
plate_xt = []
plate_yt = []
i = 0
while i < len(PLATEID_Test):
    plate_hdu  = Full_table[i]
    obj = 0
    while obj <len(plate_hdu):
        currentobj = plate_hdu[obj]
        Currentplate_z = currentobj[4]
        Currentplate_y = currentobj[2]
        Currentplate_x = currentobj[1]
        Currentplate_n = currentobj[0]
        plate_rt.append(Currentplate_z)
        plate_nt.append(Currentplate_n)
        plate_xt.append(Currentplate_x[:800])
        plate_yt.append(Currentplate_y)
        obj = obj +1
    i = i+1

plate_xt = scaler.transform(plate_xt)

In [12]:
predictions = mlp.predict(np.array(plate_xt))

In [13]:
Y_test=plate_yt
print(classification_report(Y_test,predictions))

             precision    recall  f1-score   support

          1       0.71      0.79      0.75       745
          3       0.58      0.40      0.47       291
          4       0.68      0.25      0.37        51
         30       0.64      0.68      0.66       658

avg / total       0.66      0.67      0.66      1745



In [14]:
star,star_starloc,star_lowzloc,star_galloc,star_highzloc = classification(1,Y_test,predictions) 
lowz,lowz_starloc,lowz_loc,lowz_galloc,lowz_highzloc = classification(3,Y_test,predictions)
gal,gal_starloc,gal_lowzloc,gal_galloc,gal_highzloc = classification(4,Y_test,predictions)
highz,highz_starloc,highz_lowzloc,highz_galloc,highz_highzloc = classification(30,Y_test,predictions)

In [15]:
File_Name = input("Please Enter File name: ")

Please Enter File name: 05.12.17_4


In [16]:
d = open(TrainingDir+slash+File_Name+"rebin"+bin_size+".txt", 'w')

In [17]:
t1=["Files used to train: ",np.str(Spectra_Files), "\n"]
sp= "\n"
t2 = ["Files used to test: ",np.str(Spectra_TestFiles), "\n"]
t3 = ["Number of training objects = ",np.str(len(Spectra_Files)), "\n"]
t5 = ["Number of testing objects = ",np.str(len(Spectra_TestFiles)), "\n","\n"]
t4 = ["Bin Size = ",bin_size, "\n"]
n1 = ["Structure of neural network: ", np.str(hiddenlayer_format),"\n"]
n2  = ["Backpropagation method used: ",np.str(backprop_method), "\n"]
n3  = ["Learning rate: ",np.str(lr), "\n"]
n4  = ["Activation Function: ",np.str(act), "\n"]
r1 = ["Results of Neural Network: ", "\n","\n"]
r2=["        ","       Star    z<2.1   Galaxy   z>2.1 ","\n",]
r3="Star           ",np.str(np.round(star[0]*100,2)),"%  ", np.str(np.round(star[1]*100,2)),"%  ",np.str(np.round(star[2]*100,2)),"%  ",np.str(np.round(star[3]*100,2)),"%","\n"
r4="Quasar z<2.1   ",np.str(np.round( lowz[0]*100,2)),"%  ", np.str(np.round( lowz[1]*100,2)),"%  ",np.str(np.round( lowz[2]*100,2)),"%  ",np.str(np.round( lowz[3]*100,2)),"%","\n"
r5="Galaxy         ",np.str(np.round( gal[0]*100,2)),"%  ", np.str(np.round( gal[1]*100,2)),"%  ",np.str(np.round( gal[2]*100,2)),"%  ",np.str(np.round( gal[3]*100,2)),"%","\n"
r6="Quasar z>2.1   ",np.str(np.round( highz[0]*100,2)),"%  ", np.str(np.round( highz[1]*100,2)),"%  ",np.str(np.round( highz[2]*100,2)),"%  ",np.str(np.round (highz[3]*100,2)),"%","\n"

In [18]:
d.writelines(t1)
d.writelines(sp)
d.writelines(t2)
d.writelines(sp)
d.writelines(t3)
d.writelines(t4)
d.writelines(t5)
d.writelines(sp)
d.writelines(sp)
d.writelines(n1)
d.writelines(n2)
d.writelines(n3)
d.writelines(n4)
d.writelines(sp)
d.writelines(sp)
d.writelines(r1)
d.writelines(r2)
d.writelines(r3)
d.writelines(r4)
d.writelines(r5)
d.writelines(r6)

In [19]:
d.close()